In [ ]:
import pandas as pd
import numpy as np
file_path = "ketapang-12-08-24.csv"
df = pd.read_csv(file_path, sep=";", decimal=",")
df['ID'] = range(1, len(df) + 1)

print(df.head())

Plot

In [ ]:
import folium
map = folium.Map(location=[df["Latitude"].mean(), df["Longitude"].mean()], zoom_start=8)

for i, row in df.iterrows():
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=row["ID"]
    ).add_to(map)

map


Distance between location (based on latitude and longitude)

In [ ]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers

    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)

    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

A = (-1.86393, 110.61875)
B = (-1.86332, 110.62273)
C = (-1.86254, 110.6216)

dist_AB = haversine(A[0], A[1], B[0], B[1])
dist_BC = haversine(B[0], B[1], C[0], C[1])
dist_AC = haversine(A[0], A[1], C[0], C[1])

print(f"Distance A to B: {dist_AB:.4f} km")
print(f"Distance B to C: {dist_BC:.4f} km")
print(f"Distance A to C: {dist_AC:.4f} km")

Merge the plot within 300m

In [ ]:
from math import radians, sin, cos, sqrt, atan2

# Haversine formula to calculate distance between two lat/lon in meters
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth radius in meters
    phi1, phi2 = radians(lat1), radians(lat2)
    dphi = radians(lat2 - lat1)
    dlambda = radians(lon2 - lon1)
    a = sin(dphi/2)**2 + cos(phi1) * cos(phi2) * sin(dlambda/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Convert decimal commas to points and to float
df["Latitude"] = df["Latitude"].astype(str).str.replace(",", ".").astype(float)
df["Longitude"] = df["Longitude"].astype(str).str.replace(",", ".").astype(float)

# Create groups: each cluster of points within 300m
visited = set()
clusters = []
threshold = 300  # meters

for i, row in df.iterrows():
    if i in visited:
        continue
    cluster = [i]
    visited.add(i)
    for j, row2 in df.iterrows():
        if j in visited:
            continue
        dist = haversine(row["Latitude"], row["Longitude"], row2["Latitude"], row2["Longitude"])
        if dist <= threshold:
            cluster.append(j)
            visited.add(j)
    clusters.append(cluster)

# Merge clusters: take mean of lat/lon, keep first metadata
merged = []
for cluster in clusters:
    subset = df.loc[cluster]
    avg_lat = subset["Latitude"].mean()
    avg_lon = subset["Longitude"].mean()
    merged.append({
        "Provinsi": subset.iloc[0]["Provinsi"],
        "Kab Kota": subset.iloc[0]["Kab Kota"],
        "Kecamatan": subset.iloc[0]["Kecamatan"],
        "Desa": ",".join(subset["Desa"].unique()),
        "Tanggal": subset.iloc[0]["Tanggal"],
        "Waktu": subset.iloc[0]["Waktu"],
        "Satelit": ",".join(subset["Satelit"].unique()),
        "Confidence": ",".join(subset["Confidence"].unique()),
        "Latitude": avg_lat,
        "Longitude": avg_lon,
        "Count": len(subset)  # how many merged points
    })

df_merged = pd.DataFrame(merged)
print(df_merged)

Save the merged into csv

In [ ]:
# df_merged.to_csv("ketapang-12-08-24_merged.csv", index=False)

Plot the merged

In [ ]:
import pandas as pd
import numpy as np
file_path = "ketapang-12-08-24_merged.csv"
df = pd.read_csv(file_path, sep=",", decimal=".")
df['ID'] = range(1, len(df) + 1)

print(df.head())

In [ ]:
import folium
map = folium.Map(location=[df["Latitude"].mean(), df["Longitude"].mean()], zoom_start=8)

for i, row in df.iterrows():
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=row["ID"]
    ).add_to(map)

map


Clustering with KMeans

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Select features (latitude & longitude)
X = df[["Latitude", "Longitude"]]

# Apply KMeans clustering with K=3
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df["Cluster"] = kmeans.fit_predict(X)

# Save results
# df.to_csv("ketapang-12-08-24_merged_clustered_3.csv", index=False)

print(df[["Latitude", "Longitude", "Cluster"]])

# Plot clusters
plt.figure(figsize=(8,6))
plt.scatter(df["Longitude"], df["Latitude"], c=df["Cluster"], cmap="tab10", s=100)
plt.scatter(kmeans.cluster_centers_[:,1], kmeans.cluster_centers_[:,0], 
            c="red", marker="X", s=200, label="Centroids")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.show()


Plotting the cluster

In [1]:
import folium
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors

# Load your clustered data
df = pd.read_csv("ketapang-12-08-24_merged_clustered_3.csv")

# Create base map
m = folium.Map(location=[df["Latitude"].mean(), df["Longitude"].mean()], zoom_start=8)

# Generate distinct colors for each cluster
num_clusters = df["Cluster"].nunique()
colormap = cm.get_cmap("tab10", num_clusters)  # can use 'tab20', 'Set1', etc.
norm = colors.Normalize(vmin=0, vmax=num_clusters-1)

cluster_colors = {
    cluster: colors.to_hex(colormap(norm(cluster)))
    for cluster in df["Cluster"].unique()
}

# Add markers by cluster
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=6,
        popup=(
            f"ID: {row['ID']}<br>"
            f"Cluster: {row['Cluster']}<br>"
            f"Desa: {row['Desa']}<br>"
            f"Kecamatan: {row['Kecamatan']}<br>"
            f"Satelit: {row['Satelit']}"
        ),
        color=cluster_colors[row["Cluster"]],
        fill=True,
        fill_color=cluster_colors[row["Cluster"]],
        fill_opacity=0.8
    ).add_to(m)

m


C:\Users\mrifq\AppData\Local\Temp\ipykernel_5440\2388479402.py:14: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colormap = cm.get_cmap("tab10", num_clusters)  # can use 'tab20', 'Set1', etc.


In [ ]:
import folium
import pandas as pd
import openrouteservice
from shapely.geometry import Point, MultiPoint
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import time

client = openrouteservice.Client(key='eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6IjFjMWEwMmNjMWZlYTUxMzk1OTllMWQzNDFlNmQzOTE0MjQzMjBkOGRkZTQ5MDBhNzY4MDY2NzdlIiwiaCI6Im11cm11cjY0In0=')

df = pd.read_csv("ketapang-12-08-24_merged_clustered_3.csv")

df['geometry'] = df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
gdf = df

# Compute centroids
X = df[["Latitude", "Longitude"]]
kmeans = KMeans(n_clusters=df["Cluster"].nunique(), random_state=42, n_init=10)
kmeans.fit(X)
centroids = kmeans.cluster_centers_

m = folium.Map(location=[df["Latitude"].mean(), df["Longitude"].mean()], zoom_start=8)

num_clusters = df["Cluster"].nunique()
colormap = plt.colormaps.get_cmap("tab10").resampled(num_clusters)
norm = colors.Normalize(vmin=0, vmax=num_clusters - 1)
cluster_colors = {
    cluster: colors.to_hex(colormap(norm(cluster)))
    for cluster in df["Cluster"].unique()
}

for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=6,
        color=cluster_colors[row["Cluster"]],
        fill=True,
        fill_color=cluster_colors[row["Cluster"]],
        fill_opacity=0.8,
        popup=(f"Cluster: {row['Cluster']}<br>"
               f"Desa: {row['Desa']}<br>"
               f"Kecamatan: {row['Kecamatan']}<br>"
               f"Satelit: {row['Satelit']}<br>"
               f"ID: {row['ID']}<br>")
    ).add_to(m)

# Snap centroids to road and store snapped points
snapped_road_points = []
for i, (lat, lon) in enumerate(centroids):
    if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
        print(f"Skipping invalid coordinate for centroid {i}: [{lat}, {lon}]")
        snapped_road_points.append(None)
        continue

    coords = [[lon, lat], [lon, lat]]
    try:
        snapped = client.directions(
            coordinates=coords,
            profile="driving-car",
            format="geojson",
            radiuses=[5000, 5000]
        )
        snapped_lon, snapped_lat = snapped["features"][0]["geometry"]["coordinates"][0]
        snapped_road_points.append((snapped_lat, snapped_lon))

        folium.Marker(
            location=[snapped_lat, snapped_lon],
            icon=folium.Icon(color="red", icon="truck", prefix="fa"),
            popup=(
                f"📍 Snapped Road {i}<br>"
                f"Snapped: {snapped_lat:.6f}, {snapped_lon:.6f}<br>"
                f"Original: {lat:.6f}, {lon:.6f}"
            )
        ).add_to(m)

    except Exception as e:
        print(f"Snapping failed for centroid {i}: {e}")
        snapped_road_points.append(None)

# Add depo point (only once)
depo_lat, depo_lon = -1.8295035212728696, 109.97892897528638
folium.Marker(
    location=[depo_lat, depo_lon],
    icon=folium.Icon(color="green", icon="home"),
    popup="Depo BPBD Ketapang"
).add_to(m)

for i, snapped_point in enumerate(snapped_road_points):
    if snapped_point is None:
        continue
    try:
        lat, lon = snapped_point
        coords = [[depo_lon, depo_lat], [lon, lat]]
        route = client.directions(
            coordinates=coords,
            profile="driving-car",
            format="geojson"
        )
        
        # Extract route geometry (actual road path)
        route_coords = route["features"][0]["geometry"]["coordinates"]
        route_distance = route["features"][0]["properties"]["summary"]["distance"] / 1000  # in km
        
        # Convert route coordinates to lat/lon format for folium
        route_latlons = [[coord[1], coord[0]] for coord in route_coords]  # swap lon,lat to lat,lon
        
        # Draw the actual road route (not straight line)
        folium.PolyLine(
            locations=route_latlons,  # Use actual route coordinates
            color="blue",
            weight=3,
            popup=f"Depo to Road {i} ({route_distance:.2f} km)"
        ).add_to(m)
        
        print(f"Distance from depo to snapped road {i}: {route_distance:.2f} km")
    except Exception as e:
        print(f"Failed to get route from depo to snapped road {i}: {e}")
    time.sleep(1.5)

m.save("new_forest_fire_clusters_map.html")
m

In [1]:
import folium
import pandas as pd
import openrouteservice
from shapely.geometry import Point, MultiPoint
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import time

client = openrouteservice.Client(key='eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6IjFjMWEwMmNjMWZlYTUxMzk1OTllMWQzNDFlNmQzOTE0MjQzMjBkOGRkZTQ5MDBhNzY4MDY2NzdlIiwiaCI6Im11cm11cjY0In0=')

df = pd.read_csv("ketapang-12-08-24_merged_clustered_3.csv")

df['geometry'] = df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
gdf = df

# Compute centroids
X = df[["Latitude", "Longitude"]]
kmeans = KMeans(n_clusters=df["Cluster"].nunique(), random_state=42, n_init=10)
kmeans.fit(X)
centroids = kmeans.cluster_centers_

m = folium.Map(location=[df["Latitude"].mean(), df["Longitude"].mean()], zoom_start=8)

num_clusters = df["Cluster"].nunique()
colormap = plt.colormaps.get_cmap("tab10").resampled(num_clusters)
norm = colors.Normalize(vmin=0, vmax=num_clusters - 1)
cluster_colors = {
    cluster: colors.to_hex(colormap(norm(cluster)))
    for cluster in df["Cluster"].unique()
}

for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=6,
        color=cluster_colors[row["Cluster"]],
        fill=True,
        fill_color=cluster_colors[row["Cluster"]],
        fill_opacity=0.8,
        popup=(f"Cluster: {row['Cluster']}<br>"
               f"Desa: {row['Desa']}<br>"
               f"Kecamatan: {row['Kecamatan']}<br>"
               f"Satelit: {row['Satelit']}<br>"
               f"ID: {row['ID']}<br>")
    ).add_to(m)

# Snap centroids to road and store snapped points
snapped_road_points = []
for i, (lat, lon) in enumerate(centroids):
    if not (-90 <= lat <= 90) or not (-180 <= lon <= 180):
        print(f"Skipping invalid coordinate for centroid {i}: [{lat}, {lon}]")
        snapped_road_points.append(None)
        continue

    coords = [[lon, lat], [lon, lat]]
    try:
        snapped = client.directions(
            coordinates=coords,
            profile="driving-car",
            format="geojson",
            radiuses=[5000, 5000]
        )
        snapped_lon, snapped_lat = snapped["features"][0]["geometry"]["coordinates"][0]
        snapped_road_points.append((snapped_lat, snapped_lon))

        folium.Marker(
            location=[snapped_lat, snapped_lon],
            icon=folium.Icon(color="red", icon="truck", prefix="fa"),
            popup=(
                f"📍 Snapped Road {i}<br>"
                f"Snapped: {snapped_lat:.6f}, {snapped_lon:.6f}<br>"
                f"Original: {lat:.6f}, {lon:.6f}"
            )
        ).add_to(m)

    except Exception as e:
        print(f"Snapping failed for centroid {i}: {e}")
        snapped_road_points.append(None)

# Add depo point (only once)
depo_lat, depo_lon = -1.8295035212728696, 109.97892897528638
folium.Marker(
    location=[depo_lat, depo_lon],
    icon=folium.Icon(color="green", icon="home"),
    popup="Depo BPBD Ketapang"
).add_to(m)

for i, snapped_point in enumerate(snapped_road_points):
    if snapped_point is None:
        continue
    try:
        lat, lon = snapped_point
        coords = [[depo_lon, depo_lat], [lon, lat]]
        route = client.directions(
            coordinates=coords,
            profile="driving-car",
            format="geojson"
        )
        
        # Extract route geometry (actual road path)
        route_coords = route["features"][0]["geometry"]["coordinates"]
        route_distance = route["features"][0]["properties"]["summary"]["distance"] / 1000  # in km
        
        # Convert route coordinates to lat/lon format for folium
        route_latlons = [[coord[1], coord[0]] for coord in route_coords]  # swap lon,lat to lat,lon
        
        # Draw the actual road route (not straight line)
        folium.PolyLine(
            locations=route_latlons,  # Use actual route coordinates
            color="blue",
            weight=3,
            popup=f"Depo to Road {i} ({route_distance:.2f} km)"
        ).add_to(m)
        
        print(f"Distance from depo to snapped road {i}: {route_distance:.2f} km")
    except Exception as e:
        print(f"Failed to get route from depo to snapped road {i}: {e}")
    time.sleep(1.5)

m.save("new_forest_fire_clusters_map.html")
m

Distance from depo to snapped road 0: 218.71 km
Distance from depo to snapped road 1: 107.55 km
Distance from depo to snapped road 2: 274.05 km
